In [1]:
import numpy as np
from pyscf import gto, df, scf, cc

In [53]:
a = 2 # bond length in a cluster
d = 10 # distance between each cluster
unit = 'b' # unit of length
na = 2  # size of a cluster (monomer)
nc = 1 # set as integer multiple of monomers
spin = 0 # spin per monomer
frozen = 0 # frozen orbital per monomer
elmt = 'O'
basis = 'sto6g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms,basis=basis,spin=spin*nc,unit=unit,verbose=4)
mol.build()

mf = scf.RHF(mol)
emf = mf.kernel()

mycc = cc.CCSD(mf)
eccsd = mycc.kernel()[0]

print('SCF energy: ', emf)
print('CCSD energy: ', eccsd)

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-37-generic', version='#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Thu Jan  8 14:23:15 2026
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 16
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = b
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr 

In [54]:
def compress_symmetric(A):
    """
    Compress symmetric matrix A (n x n) into 1D array
    storing the upper triangle (including diagonal).
    """
    n = A.shape[0]
    assert A.shape == (n, n)
    idx = np.triu_indices(n)
    return A[idx]

def compress_chol(chol):
    (nchol,nao,_) = chol.shape
    comp_chol = np.empty((nchol,nao*(nao+1)//2))
    for n in range(nchol):
        comp_chol[n,:] = compress_symmetric(chol[n])
    return comp_chol

In [ ]:
from pyscf import ao2mo, lib
from ad_afqmc import pyscf_interface
einsum = lib.einsum

In [79]:
nao = mol.nao
eri_s1 = ao2mo.restore('s1',mf._eri,nao)
eri_s4 = ao2mo.restore('s4',mf._eri,nao)
print(eri_s1.shape)
print(eri_s4.shape)

(10, 10, 10, 10)
(55, 55)


In [90]:
chol_s4 = pyscf_interface.modified_cholesky(eri_s4,1e-8) #chunked_cholesky(mol, 1e-7)
nao = mol.nao
print(chol_s4.shape)
# chol = chol.reshape(-1, nao, nao)
# print(chol.shape)
# print(chol)
# comp_chol = compress_chol(chol)
# print(chol.shape)
# print(comp_chol.shape)
eri_chol_s4 = einsum('gp,gq->pq', chol_s4, chol_s4)
print(abs(eri_s4-eri_chol_s4).max())

(51, 55)
4.024746470560753e-09


In [92]:
mfdf = scf.RHF(mol).density_fit()
mfdf._cderi = chol_s4
emfdf = mfdf.kernel()

myccdf = cc.CCSD(mfdf)
eccsddf = myccdf.kernel()[0]

print(f'SCF energy = {emf}')
print(f'DF-SCF energy = {emfdf}, error = {emfdf-emf:.2e}')
print(f'CCSD energy = {eccsd}')
print(f'DF-CCSD energy = {eccsddf}, error = {eccsddf-eccsd:.2e}')



******** <class 'pyscf.df.df_jk.DFRHF'> ********
method = DFRHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = False
chkfile to save SCF result = /tmp/tmp6szbepiq
max_memory 4000 MB (current use 333 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
init E= -149.699418954668

WARN: HOMO -0.063797817193913 == LUMO -0.0637978171939123

cycle= 1 E= -148.878147174953  delta_E= 0.821  |g|= 0.127  |ddm|= 2.54
  HOMO = -0.298755300436419  LUMO = 0.239448510628331
cycle= 2 E= -148.881437947622  delta_E= -0.00329  |g|= 0.0226  |ddm|= 0.172
  HOMO = -0.296572234498982  LUMO = 0.241631576565772
cycle= 3 E= -148.881602581751  delta_E= -0.000165  |g|= 0.00275  |ddm|= 0.0451
  HOMO = -0.296224760498961  LUMO = 0.241979050565787
cycle= 4 E= -148.881604803239  delta_E= -2.22e-06  |g|= 9.67e-07  |d